# Inference Pipeline
Predicts delays for the next hour given delays from the last three 30 minutes

### Imports

In [1]:
from datetime import date
import hopsworks
import os
import pandas as pd
from pathlib import Path
import sys
import joblib
import time

root_dir = Path().absolute()
# Strip subdirectories if the notebook started in any
if root_dir.parts[-1:] == ('pipeline',):
    root_dir = Path(*root_dir.parts[:-1])
if root_dir.parts[-1:] == ('src',):
    root_dir = Path(*root_dir.parts[:-1])
root_dir = str(root_dir) 

os.chdir(root_dir)
print(f"Root dir: {Path.cwd()}")

from src.data_utils.plots import *

Root dir: /Users/serkan/ID2223-project


### Connect to Hopsworks

In [2]:
# Enter the project name if the project in Hopsworks is not your main project
#project_name = None
project_name = 'metro_delay_prediction'
max_retries = 3
for i in range(max_retries):
    try:
        if project_name:
            project = hopsworks.login(project=f'{project_name}')
        else:
            project = hopsworks.login()
        fs = project.get_feature_store()
        break
    except Exception as e:
        print(f'Error {e}, retrying in 1 second')
        time.sleep(1)
else:
    raise RuntimeError(f'Failed to connect to hopsworks')

2026-01-05 11:18:40,369 INFO: Initializing external client
2026-01-05 11:18:40,369 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-05 11:18:40,950 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2026-01-05 11:18:41,818 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1330333


### Retrieve Model from Model Registry and load model + artifacts

In [3]:
max_retries = 3
for i in range(max_retries):
    try:
        mr = project.get_model_registry()
        retrieved_model = mr.get_model(name='MLP', version=1)

        model_dir = retrieved_model.download()

        # fv = retrieved_model.get_feature_view()
        # saved_dir = retrieved_model.download()
        MLP = joblib.load(f'{model_dir}/model.joblib')
        line_encoder = joblib.load(f'{model_dir}/line_encoder.pkl')
        day_encoder = joblib.load(f'{model_dir}/day_encoder.pkl')
        break
    except Exception as e:
        print(f'Error {e}, retrying in 1 second')
        time.sleep(1)
else:
    raise RuntimeError(f'Failed to get model and artifacts.')

Downloading: 0.000%|          | 0/125071 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/517 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/21293 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/579 elapsed<00:00 remaining<?

### Fetch features and take most recent data

In [4]:
max_retries = 3
for i in range(max_retries):
    try:
        features_fg = fs.get_feature_group(
            name="delay_features_fg",
            version=1
        )
        features_df = features_fg.read(online=True)
        break
    except Exception as e:
        print(f'Error {e}, retrying in 1 second')
        time.sleep(1)
else:
    raise RuntimeError(f'Failed to get features')

In [5]:
latest_df = features_df.sort_values("timestamp").groupby("line").tail(1).reset_index(drop=True)
print(latest_df)

            timestamp        timestamp_str          line   delay_60  \
0 2026-01-05 10:32:00  2026-01-05T10:32:00  Gröna linjen  47.156250   
1 2026-01-05 10:32:00  2026-01-05T10:32:00    Blå linjen -31.384615   
2 2026-01-05 10:32:00  2026-01-05T10:32:00   Röda linjen -19.321429   

    delay_30  delay_current  
0  39.125000      57.032258  
1 -23.166667     -34.916667  
2 -18.200000     -12.320000  


### Encode non-numerical features and set up data for inference

In [6]:
latest_df["day"] = latest_df["timestamp"].dt.day_name()
latest_df["day_encoded"] = day_encoder.transform(latest_df["day"])
latest_df["line_encoded"] = line_encoder.transform(latest_df["line"])
print(latest_df.head())

2026-01-05 11:19:39,555 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-05 11:19:39,555 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-05 11:19:39,556 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-05 11:19:39,557 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

            timestamp        timestamp_str          line   delay_60  \
0 2026-01-05 10:32:00  2026-01-05T10:32:00  Gröna linjen  47.156250   
1 2026-01-05 10:32:00  2026-01-05T10:32:00    Blå linjen -31.384615   
2 2026-01-05 10:32:00  2026-01-05T10:32:00   Röda linjen -19.321429   

    delay_30  delay_current     day

In [7]:
inference_df = latest_df[["line_encoded", "day_encoded", "delay_60", "delay_30", "delay_current"]].copy()
X = inference_df.dropna(subset=["delay_60", "delay_30", "delay_current"]).reset_index(drop=True)
print(X)

   line_encoded  day_encoded   delay_60   delay_30  delay_current
0             1            1  47.156250  39.125000      57.032258
1             0            1 -31.384615 -23.166667     -34.916667
2             2            1 -19.321429 -18.200000     -12.320000


### Make predictions

In [8]:
preds = MLP.predict(X)
print(preds)

2026-01-05 11:19:58,832 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-05 11:19:58,833 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-05 11:19:58,834 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

[ 57.08550162 -28.68355666 -15.06218126]


In [9]:
inference_df["line"] = line_encoder.inverse_transform(inference_df["line_encoded"])
inference_df["prediction"] = preds
inference_df["timestamp"] = latest_df["timestamp"]
inference_df["timestamp_pk"] = inference_df["timestamp"].dt.strftime("%Y-%m-%dT%H:%M:%S")
inference_df["delay_hind"] = latest_df["delay_current"]
result = inference_df[["line", "timestamp", "timestamp_pk", "prediction", "delay_hind"]]
print(result)

2026-01-05 11:20:03,781 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-05 11:20:03,782 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

           line           timestamp         timestamp_pk  prediction  \
0  Gröna linjen 2026-01-05 10:32:00  2026-01-05T10:32:00   57.085502   
1    Blå linjen 2026-01-05 10:32:00  2026-01-05T10:32:00  -28.683557   
2   Röda linjen 2026-01-05 10:32:00  2026-01-05T10:32:00  -15.062181   

   delay_hind  
0   57.032258  
1  -34.916667  
2  -12.320000  


### Upload predictions to hopsworks

In [ ]:
max_retries = 3
for i in range(max_retries):
    try:
        monitor_fg = fs.get_or_create_feature_group(
            name='sl_prediction',
            description='SL metro lines prediction monitoring',
            version=1,
            primary_key=['line', 'timestamp_pk'],
            event_time='timestamp',
        )
        monitor_fg.insert(result, wait=True)
        break
    except Exception as e:
        print(f'Error {e}, retrying in 1 second')
        time.sleep(1)
else:
    raise RuntimeError(f'Failed to insert to monitor feature group')

### Get all predictions and update plots

In [ ]:
max_retries = 3
for i in range(max_retries):
    try:
        predictions_fg = fs.get_feature_group(
            name="sl_prediction",
            version=1
        )
        predictions_df = predictions_fg.read()
        break
    except Exception as e:
        print(f'Error {e}, retrying in 1 second')
        time.sleep(1)
else:
    raise RuntimeError(f'Failed to read predictions feature group')

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.12s) 


In [ ]:
images_path = Path('docs/images/')
plot_metro_delay_predictions(predictions_df, images_path, hindcast=True)
plot_metro_delay_predictions(predictions_df, images_path, hindcast=False)

2026-01-03 19:08:09,735 WARNING: DeprecationWarning: backend2gui is deprecated since IPython 8.24, backends are managed in matplotlib and can be externally registered.

